In [1]:
!make

ModuleNotFoundError: No module named 'qsimcirq'

In [ ]:
import cirq
import qsimcirq
from cirq.ops import *

In [ ]:
import numpy as np 
import time
import matplotlib.pyplot as plt

In [ ]:
max_depth = 20
shot_count = 1

def getSimulator():
    #return cirq.Simulator()
    return qsimcirq.QSimSimulator()

def entangler(circuit,qubits):
    n = len(qubits)
    for i in range(n-1):
        circuit.append(rx(3)(qubits[i+1]).controlled_by(qubits[i]))

def entanglerReverse(circuit,qubits):
    n = len(qubits)
    for i in range(n):
        target = n-i-1
        if i != target:
            circuit.append(rx(np.pi/7)(qubits[target]).controlled_by(qubits[i]))

def entanglerCX(circuit,qubits):
    n = len(qubits)
    for i in range(n-1):
        circuit.append(CX(qubits[i],qubits[i+1]))

def entanglerReverseCX(circuit,qubits):
    n = len(qubits)
    for i in range(n):
        target = n-i-1
        if i != target:
            circuit.append(CX(qubits[i],qubits[target]))

def Rx(citrcuit,qubits):
    for q in qubits:
        citrcuit.append(rx(np.pi/3)(q))

def Ry(citrcuit,qubits):
    for q in qubits:
        citrcuit.append(ry(np.pi/5)(q))

def XY(citrcuit,qubits):
    for q in qubits:
        citrcuit.append(X(q))
        citrcuit.append(Y(q))

In [ ]:
def bench(simulator, max_depth, exec_func):
    i=2
    j=2
    depth=[]
    for i in range(2,max_depth):
        duration=0
        while (duration<0.50) and j<101:
            test_qc=test(i,j)
            start_time = time.time()
            a = exec_func(simulator, test_qc)
            a._measurements.get('0',0)
            duration=(time.time() - start_time)
            j=j+1
        depth.append(j-1)
        j=2
    return depth

def scor(depth):
    sc=0
    i=2
    for x in depth:
        sc=sc+depth*i
        i=i+1
    return sc

In [ ]:
def test(i,j):
    qubits = cirq.LineQubit.range(i)
    qc = cirq.Circuit()
    for k in range(j):
        Rx(qc,qubits)
        entangler(qc,qubits)
        Ry(qc,qubits)
        entanglerReverse(qc,qubits)
    for q in qubits:
        qc.append(measure(q))
    return qc

In [ ]:
def exec_func(sim, circ):
    return sim.run(circ)

simulator=getSimulator()
depth=bench(simulator,max_depth, exec_func)
print(depth)

In [ ]:
simulator_name='Qiskit Aer-qasm_simulator'

fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
langs = [i for i in range(2,max_depth)]
ax.bar(langs,depth)
ax.set_title(' MaxDepth(nr,Qubits) ['+ simulator_name+']')
ax.set_xlabel('nr. Qubits')
ax.set_ylabel('Depth (RxRy_Cx)')
plt.show()